# Providing Conversation History to Amazon Bedrock Agents

In this notebook, we will create an Amazon Bedrock Agent and understand how to initialize the Agent with Conversation History. This feature enables you to continue a conversation with an Agent even if the session has expired. Additionally, we will explore the benefits of overriding Conversation History at each turn of Agent invocation, which can lead to lower latency and reduced cost. 

### 1- Initializing Amazon Bedrock Agent with Conversation History
### 2- Overriding Conversation History for Lower Input Token Consumption

In this example we will create a restaurant assistant agent that connects with a Knowledge Base for Amazon Bedrock containing the restaurant's different menus. This agent also connects to an action group that provides functionalities for handling the table booking in this restaurant.

![](./architecture.png)

## Prerequisites
Before starting, let's update the boto3 packages to ensure we have the latest version

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install -U -r requirements.txt

Let's now create the boto3 clients for the required AWS services

In [ ]:
import boto3


print(boto3.__version__)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")

In [7]:
import time
import uuid
import json
import logging
import pprint
from termcolor import colored

from create_agent import create_restaurant_agent, cleanup, create_kb
from utility import create_latency_boxplot, create_token_boxplot, last_message_analytics

In [8]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Create Restaurant Assistant Agent

In [ ]:
kb_id, knowledge_base = create_kb()

In [ ]:
agent_id, alias_id, lambda_function, agent_action_group_response = create_restaurant_agent(kb_id)

## Invoke Agent - Utility

#### The conversationHistory can be passed into the sessionState parameter of the invokeAgent API in the following format:

```json
sessionState={
    'conversationHistory': {
        'messages': [
            {
                'content': [
                    {
                        'text': 'string'
                    },
                ],
                'role': 'user'|'assistant'
            },
            // Additional message objects...
        ]
    },
}
```

#### Each message object in the messages array represents a single turn in the conversation, where the content field contains the message text, and the role field specifies whether the message is from the 'user' or the 'assistant'.

#### When passing the conversationHistory in the sessionState, it appends the provided history to any existing messages in the current session.

In [33]:
def invokeAgent(query, session_id, enable_trace=False, trace_level="core", session_state=dict(), end_session=False):
    agent_answer = str()
    input_tokens = 0
    output_tokens = 0
    start_time = time.time()

    if trace_level not in ["core", "all"]:
        trace_level = "core"

        print(colored(f"Defaulting to trace level: {trace_level}", "red"))

    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )
    
    if enable_trace and trace_level == "all":
        logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace and trace_level == "all":
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                
                # End event indicates that the request finished successfully
            if 'trace' in event:
                if enable_trace:
                    if trace_level == "all":
                        logger.info(json.dumps(event['trace'], indent=2))
                    
                    trace = event["trace"]["trace"]

                    if "orchestrationTrace" in trace:
                        orchestrationTrace = trace["orchestrationTrace"]

                        if "modelInvocationOutput" in orchestrationTrace:
                            input_tokens += orchestrationTrace["modelInvocationOutput"]["metadata"]["usage"]["inputTokens"]
                            output_tokens += orchestrationTrace["modelInvocationOutput"]["metadata"]["usage"]["outputTokens"]

                    if "postProcessingTrace" in trace:
                        postProcessingTrace = trace["postProcessingTrace"]
                        
                        if "modelInvocationOutput" in postProcessingTrace:
                            input_tokens += postProcessingTrace["modelInvocationOutput"]["metadata"]["usage"]["inputTokens"]
                            output_tokens += postProcessingTrace["modelInvocationOutput"]["metadata"]["usage"]["outputTokens"]

                    if "preProcessingTrace" in trace:
                        preProcessingTrace = trace["preProcessingTrace"]
                        
                        if "modelInvocationOutput" in preProcessingTrace:
                            input_tokens += preProcessingTrace["modelInvocationOutput"]["metadata"]["usage"]["inputTokens"]
                            output_tokens += preProcessingTrace["modelInvocationOutput"]["metadata"]["usage"]["outputTokens"]

                    if "routingClassifierTrace" in trace:
                        routingClassifierTrace = trace["routingClassifierTrace"]
                        
                        if "modelInvocationOutput" in routingClassifierTrace:
                            input_tokens += routingClassifierTrace["modelInvocationOutput"]["metadata"]["usage"]["inputTokens"]
                            output_tokens += routingClassifierTrace["modelInvocationOutput"]["metadata"]["usage"]["outputTokens"]
                        
                        
                    
    except Exception as e:
        raise Exception("unexpected event.", e)

    end_time = time.time()

    execution_time = end_time - start_time

    if not end_session:
        print(colored(f"Agent took {execution_time} seconds to execute", "yellow"))
        print(colored(f"Agent took Input Tokens: {input_tokens}  and Output Tokens: {output_tokens}  to execute", "yellow"))
    
    return agent_answer, execution_time, input_tokens, output_tokens

## Initializing Amazon Bedrock Agent with Conversation History

#### In this section we will initialize the agent with the following conversation history. 

```
User: What are the starters in the childrens menu?
```
```
Assistant: According to the knowledge base, the children's menu at The Regrettable Experience does not have a separate "starters" section. However, some of the entree options could be considered starters or appetizers, such as:
            - Chicken nuggets (allergens: gluten, possible soy)
            - Macaroni and cheese (allergens: dairy, gluten)  
            - Mini cheese quesadillas (allergens: dairy, gluten)
            - Peanut butter and banana sandwich (allergens: nuts, gluten)
            - Veggie pita pockets (allergens: gluten, possible soy)
            - Mini cheeseburgers (allergens: dairy, gluten)
            - Fish sticks (allergens: gluten, possible soy)
            - Grilled cheese sandwich (allergens: dairy, gluten)
            - Spaghetti with marinara sauce (allergens: gluten)
```

#### To invoke the agent with conversation history, you need to pass it in the following format in the 'sessionState' parameter of the 'InvokeAgent' API request.

In [34]:
# Start a new sesssion
session_id:str = str(uuid.uuid1())

# conversation history
conversation_history = [
    {"content": [{"text": """
        According to the knowledge base, the children's menu at The Regrettable Experience does not have a separate "starters" section. However, some of the entree options could be considered starters or appetizers, such as:
            
            - Chicken nuggets (allergens: gluten, possible soy)
            - Macaroni and cheese (allergens: dairy, gluten)  
            - Mini cheese quesadillas (allergens: dairy, gluten)
            - Peanut butter and banana sandwich (allergens: nuts, gluten)
            - Veggie pita pockets (allergens: gluten, possible soy)
            - Mini cheeseburgers (allergens: dairy, gluten)
            - Fish sticks (allergens: gluten, possible soy)
            - Grilled cheese sandwich (allergens: dairy, gluten)
            - Spaghetti with marinara sauce (allergens: gluten)
        """}], 
     "role": "assistant"},
    {"content": [{"text": "What are the starters in the childrens menu?"}], "role": "user"},
]

In [ ]:
query = "Hi, I am Anna. I want to create a booking for 2 people, at 8pm on the 5th of May 2024."

session_state = {'conversationHistory': {'messages': conversation_history}}

response, _, _, _ = invokeAgent(query=query, session_id=session_id, enable_trace=True, session_state=session_state) # pass the conversation history in session_state

conversation_history.insert(0, {"content": [{"text": query}], "role": "user"})
conversation_history.insert(0, {"content": [{"text": response}], "role": "assistant"})

print(response)

#### Terminate the current agent session.

In [ ]:
invokeAgent(query="End", session_id=session_id, enable_trace=True, session_state=dict(), end_session=True)

In [ ]:
print(conversation_history)

#### We can now initialize a new session and invoke the agent by passing the conversation history in the 'sessionState' parameter. This enables executing queries like 'Can you get information about my most recent booking?' by utilizing the context from the previous conversation.

In [ ]:
session_id:str = str(uuid.uuid1()) # Start a new session

query = "Can you get information about the most recent booking?"

session_state = {'conversationHistory': {'messages': conversation_history}} # pass the conversation history in session_state

response, _, _, _ = invokeAgent(query=query, session_id=session_id, enable_trace=False, session_state=session_state)

print(response)

conversation_history.insert(0, {"content": [{"text": query}], "role": "user"})
conversation_history.insert(0, {"content": [{"text": response}], "role": "assistant"})

In [ ]:
print(conversation_history)

## 2- Overriding Conversation History for Lower Input Token Consumption

#### Within a single session of an agent, the conversation history grows larger with each 'invokeAgent' API call. For example, if we execute the query 'What are the starters in the Children's Menu?', the agent's conversation history after processing this query would be:


```json
{
  "messages": [
    {
      "content": "What are the starters in Childrens Menu\"",
      "role": "user"
    },
    {
      "content": "<thinking>The user is asking about the starters available in the children's menu. To find this information, I will search the knowledge base.</thinking><function_calls><invoke><tool_name>GET::x_amz_knowledgebase_XXXXXXX::Search</tool_name><parameters><searchQuery>starters in children's menu</searchQuery></parameters></invoke></function_calls>",
      "role": "assistant"
    },
    {
      "content": "<function_results><result><tool_name>GET::x_amz_knowledgebase_XXXXXX::Search</tool_name><stdout>Here are search results in numbered order:\n<search_result>\n    <answer_part>\n    <text>\n        The Regrettable Experience \\u2013 Week  specials    Monday Lunch Specials    \\u2022 Soup of the Day: Hearty Lentil Soup (Vegetarian/Vegan)   \\u2022 Salad: Grilled Chicken Caesar Salad     Allergens: Dairy (in dressing), Gluten (in croutons), Eggs (in dressing)   \\u2022 Sandwich: BLT Wrap with Avocado     Allergens: Gluten (in wrap)     Tuesday Lunch Special     \\u2022 Soup of the Day: Creamy Tomato Basil (Vegetarian)   Allergens: Dairy     \\u2022 Salad: Strawberry Spinach Salad with Balsamic Vinaigrette   Allergens: Potential Tree Nuts (in vinaigrette)   Quiche: Broccoli and Cheddar Quiche   Allergens: Dairy, Eggs, Gluten (in crust)     Wednesday Lunch Special     \\u2022 Soup of the Day: Chicken Tortilla Soup   Allergens: Dairy, Gluten (in tortilla strips)     \\u2022 Salad: Greek Salad with Feta and Olives   Allergens: Dairy (in feta)     \\u2022 Flatbread: Margherita Flatbread   Allergens: Dairy, Gluten     Thursday Lunch Special     \\u2022 Soup of the Day: Black Bean Soup (Vegetarian/Vegan)   \\u2022 Salad: Southwest Chicken Salad with Avocado Ranch     Allergens: Dairy (in dressing), Eggs (in dressing)   \\u2022 Sandwich: Turkey Bacon Avocado Melt     Allergens: Dairy, Gluten (in bread)     Friday Lunch Special     \\u2022 Soup of the Day: New England Clam Chowder   Allergens: Dairy, Shellfish          \\u2022 Salad: BBQ Chicken Salad with Ranch   Allergens: Dairy (in dressing), Eggs (in dressing)     \\u2022 Burger: Black Bean Veggie Burger with Sweet Potato Fries   Allergens: Gluten (in bun), Soy (in veggie patty)         Please let your server know about any food allergies or dietary restrictions.\n    </text>\n    <sources>\n        <source>1</source>\n    </sources>\n</answer_part>\n<answer_part>\n    <text>\n        The Regrettable Experience -- Dinner Menu Entrees:   1. BUFFALO CHICKEN WINGS   \\u25CF   \\u25CF   Description: Classic buffalo wings served with celery sticks and blue cheese dressing.   Allergens: Dairy (in blue cheese dressing), Gluten (in the coating), possible Soy (in the sauce).   2. SHRIMP AND GRITS   \\u25CF   \\u25CF   Description: Succulent shrimp saut\\u00E9ed in a flavorful sauce, served over creamy cheese grits.   Allergens: Dairy (in the cheese grits).   3. CAPRESE SALAD STUFFED AVOCADO   \\u25CF   \\u25CF   Description: Avocado halves filled with a fresh caprese salad of tomatoes, mozzarella, and basil, drizzled with balsamic glaze.   Allergens: Dairy (in mozzarella), potential Tree Nuts (if using balsamic glaze with nuts).   4. LOADED POTATO SKINS   \\u25CF   \\u25CF   Description: Crispy potato skins filled with melted cheddar cheese, bacon bits, green onions, and a dollop of sour cream.   Allergens: Dairy (in cheese and sour cream), possible Soy (in bacon bits).   5. VEGETARIAN CHILI   \\u25CF   \\u25CF   Description: Hearty chili made with a variety of beans, vegetables, and spices, topped with shredded cheese and chopped green onions.   Allergens: None in the basic recipe, but be mindful of specific dietary restrictions.   Mains:   1. GRILLED BBQ CHICKEN   \\u25CF   \\u25CF   Description: Juicy grilled chicken breasts slathered in smoky barbecue sauce, served with coleslaw and baked beans.   Allergens: Possible Gluten (in barbecue sauce), potential Soy (in marinade).   2. SOUTHERN FRIED CATFISH   \\u25CF   \\u25CF   Description: Cornmeal-crusted catfish fillets, fried to perfection and served with hushpuppies and tartar sauce.   Allergens: Gluten (in the breading).   3. BBQ PULLED PORK SANDWICHES   \\u25CF   \\u25CF   Description: Slow-cooked pulled pork in tangy barbecue sauce, piled high on a bun and served with coleslaw.   Allergens: Possible Gluten (in barbecue sauce, bun), potential Soy (in sauce).   4.\n    </text>\n    <sources>\n        <source>2</source>\n    </sources>\n</answer_part>\n<answer_part>\n    <text>\n        Allergens: Gluten (in the breading).   3. BBQ PULLED PORK SANDWICHES   \\u25CF   \\u25CF   Description: Slow-cooked pulled pork in tangy barbecue sauce, piled high on a bun and served with coleslaw.   Allergens: Possible Gluten (in barbecue sauce, bun), potential Soy (in sauce).   4. STEAK WITH GARLIC BUTTER        \\u25CF   \\u25CF   Description: Grilled sirloin steak seasoned with garlic butter, accompanied by loaded mashed potatoes and saut\\u00E9ed vegetables.   Allergens: Dairy (in garlic butter), potential Soy (in marinade).   5. VEGETABLE STIR-FRY WITH TOFU   \\u25CF   \\u25CF   Description: Stir-fried tofu and a colorful medley of vegetables in a savory soy-ginger sauce, served over jasmine rice.   Allergens: Soy (in tofu and sauce).   Desserts:   1. CLASSIC NEW YORK CHEESECAKE   \\u25CF   \\u25CF   Description: Creamy cheesecake with a graham cracker crust, topped with a choice of fruit compote or chocolate ganache.   Allergens: Dairy (in cheesecake), Gluten (in the crust).   2. APPLE PIE A LA MODE   \\u25CF   \\u25CF   Description: Warm apple pie with a flaky crust, served with a scoop of vanilla ice cream and a drizzle of caramel sauce.   Allergens: Dairy (in ice cream), Gluten (in pie crust).   3. CHOCOLATE LAVA CAKE   \\u25CF   \\u25CF   Description: Rich and gooey chocolate cake with a molten center, dusted with powdered sugar and served with a scoop of raspberry sorbet.   Allergens: Dairy (in cake), possible Gluten (depending on recipe), potential Soy (in chocolate).   4. PECAN PIE BARS   \\u25CF   \\u25CF   Description: Buttery shortbread crust topped with a gooey pecan filling, cut into bars for easy serving.   Allergens: Dairy (in crust), Nuts (in pecan filling), Gluten (in crust).   5. BANANA PUDDING PARFAIT   \\u25CF   \\u25CF   Description: Layers of vanilla pudding, sliced bananas, and vanilla wafers, topped with whipped cream and a sprinkle of crushed nuts.   Allergens: Dairy (in pudding), Gluten (in wafers).\n    </text>\n    <sources>\n        <source>3</source>\n    </sources>\n</answer_part>\n<answer_part>\n    <text>\n        Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   4. SPAGHETTI WITH MARINARA SAUCE   \\u25CF   \\u25CF   \\u25CF   Description: Kid-friendly spaghetti noodles topped with tomato marinara sauce.   Allergens: Gluten.   Suitable for Vegetarians: Yes   5. MINI PITA PIZZA   \\u25CF   \\u25CF   \\u25CF   Description: Small pita bread topped with tomato sauce, cheese, and favorite toppings.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   Desserts:   1. MINI ICE CREAM SUNDAE   \\u25CF   \\u25CF   \\u25CF   Description: A scoop of vanilla ice cream with chocolate sauce and rainbow sprinkles.   Allergens: Dairy.   Suitable for Vegetarians: Yes   2. FRUIT KABOBS   \\u25CF   \\u25CF   \\u25CF   Description: Fresh fruit chunks on skewers for a fun and healthy dessert.   Allergens: None.   Suitable for Vegetarians: Yes   3. CHOCOLATE CHIP COOKIE BITES   \\u25CF   \\u25CF   \\u25CF   Description: Bite-sized chocolate chip cookies.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   4. BANANA SPLIT   \\u25CF   \\u25CF   \\u25CF   Description: Sliced banana with scoops of strawberry, chocolate, and vanilla ice cream, topped with whipped cream and a cherry.   Allergens: Dairy.   Suitable for Vegetarians: Yes   5. JELLO CUPS   \\u25CF   \\u25CF   Description: Colorful and wiggly gelatin cups.   Allergens: None.        \\u25CF Suitable for Vegetarians: Yes\n    </text>\n    <sources>\n        <source>4</source>\n    </sources>\n</answer_part>\n<answer_part>\n    <text>\n        The Regrettable Experience \\u2014 Children's Menu Entrees:   1. CHICKEN NUGGETS   \\u25CF   \\u25CF   \\u25CF   Description: Crispy chicken nuggets served with a side of ketchup or ranch dressing.   Allergens: Gluten (in the coating), possible Soy.   Suitable for Vegetarians: No   2. MACARONI AND CHEESE   \\u25CF   \\u25CF   \\u25CF   Description: Classic macaroni pasta smothered in creamy cheese sauce.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   3. MINI CHEESE QUESADILLAS   \\u25CF   \\u25CF   \\u25CF   Description: Small flour tortillas filled with melted cheese, served with a mild salsa.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   4. PEANUT BUTTER AND BANANA SANDWICH   \\u25CF   \\u25CF   \\u25CF   Description: Peanut butter and banana slices on whole wheat bread.   Allergens: Nuts (peanut), Gluten.   Suitable for Vegetarians: Yes (if using vegetarian peanut butter)   5. VEGGIE PITA POCKETS   \\u25CF   \\u25CF   \\u25CF   Description: Mini whole wheat pita pockets filled with hummus, cucumber, and cherry tomatoes.   Allergens: Gluten, possible Soy.   Suitable for Vegetarians: Yes   Mains:   1. MINI CHEESEBURGERS   \\u25CF   \\u25CF   \\u25CF   Description: Small beef patties topped with cheese, served on mini buns.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: No   2. FISH STICKS   \\u25CF   \\u25CF   \\u25CF   Description: Breaded fish sticks served with tartar sauce.   Allergens: Gluten (in the breading), possible Soy.   Suitable for Vegetarians: No        3. GRILLED CHEESE SANDWICH   \\u25CF   \\u25CF   \\u25CF   Description: Melted cheese between slices of buttered bread, grilled to perfection.   Allergens: Dairy, Gluten.   Suitable for Vegetarians: Yes   4. SPAGHETTI WITH MARINARA SAUCE   \\u25CF   \\u25CF   \\u25CF   Description: Kid-friendly spaghetti noodles topped with tomato marinara sauce.   Allergens: Gluten.   Suitable for Vegetarians: Yes   5.\n    </text>\n    <sources>\n        <source>5</source>\n    </sources>\n</answer_part>\n</search_result>\n</stdout></result></function_results>",
      "role": "user"
    },
      {
        "content": "<answer>Based on the search results, the starters or appetizers listed in the children's menu are:\n\n1. Chicken nuggets - Crispy chicken nuggets served with ketchup or ranch dressing. Contains gluten and possible soy.\n\n2. Mini cheese quesadillas - Small flour tortillas filled with melted cheese, served with mild salsa. Contains dairy and gluten.\n\n3. Veggie pita pockets - Mini whole wheat pita pockets filled with hummus, cucumber, and cherry tomatoes. Contains gluten and possible soy. Suitable for vegetarians.</answer>",
        "role": "assistant"
    }
  ]
}
```

#### We can instead pass a simplified version of conversation history in the 'sessionState' parameter of the 'invokeAgent' API request. This approach ensures that we retain relevant content while decreasing the input token consumption. 

```json
{
    [
        {"content": [{"text": "According to the knowledge base, the children's menu at The Regrettable Experience does not have a separate starters section. However, some of the entree options could be considered starters or appetizers, such as:
                    
                    - Chicken nuggets (allergens: gluten, possible soy)
                    - Macaroni and cheese (allergens: dairy, gluten)  
                    - Mini cheese quesadillas (allergens: dairy, gluten)
                    - Peanut butter and banana sandwich (allergens: nuts, gluten)
                    - Veggie pita pockets (allergens: gluten, possible soy)
                    - Mini cheeseburgers (allergens: dairy, gluten)
                    - Fish sticks (allergens: gluten, possible soy)
                    - Grilled cheese sandwich (allergens: dairy, gluten)
                    - Spaghetti with marinara sauce (allergens: gluten)
                "}], 
             "role": "assistant"},
            {"content": [{"text": "What are the starters in the childrens menu?"}], "role": "user"},
    ]
}
```

In [19]:
messages = [
    "What are the entree in Childrens Menu",
    "Hi, I am Aku. I want to create a booking for 3 people, at 8pm on the 8th of June 2025.",
    "I want to get the information about my recent booking",
    "I want to delete the recent booking"
]

### Approach - 1: Using a single session and maintaining no conversation history

#### Let's first invoke the agent using a single agent session, and take note of the input and output token counts at each turn of the conversation.

In [ ]:
session_id:str = str(uuid.uuid1())
for message in messages:
    response, _, _, _ = invokeAgent(query=message, session_id=session_id, enable_trace=True, session_state=dict())

    print(colored(f"User: {message}", "blue"))
    print(colored(f"Assistant: {response}", "green"))

    print(colored(f"Sleeping for 15 seconds.", "red"))
    time.sleep(15)

### Approach - 2: Using Coversation History

#### Now, let's invoke the agent using a new session at each turn of the conversation, while passing the stored conversation history in the 'sessionState' parameter. Take note of the input and output token counts in this case.

In [ ]:
conversation_history = list()

for message in messages:
    session_id:str = str(uuid.uuid1())
    session_state = {'conversationHistory': {'messages': conversation_history}}
    response, _, _, _ = invokeAgent(query=message, session_id=session_id, enable_trace=True, session_state=session_state)

    print(colored(f"User: {message}", "blue"))
    print(colored(f"Assistant: {response}", "green"))

    conversation_history.insert(0, {"content": [{"text": query}], "role": "user"})
    conversation_history.insert(0, {"content": [{"text": response}], "role": "assistant"})
    
    invokeAgent(query="end", session_id=session_id, enable_trace=False, session_state=dict(), end_session=True)

    print(colored(f"Sleeping for 15 seconds.", "red"))

    time.sleep(15)

#### Compare the input and output token counts between the two approaches. You will notice significantly higher input token consumption in the first approach. Providing a simplified version of conversation history at each agent invocation helps reduce the input token consumption.

## Approach 1 vs Approach 2 - Latency and Token Consumption

In [39]:
messages = [
    "What are the entree in Childrens Menu",
    "What are the weekly specials",
    "Hi, I am Aku. I want to create a booking for 3 people, at 8pm on the 8th of June 2025.",
    "I want to get the information about my recent booking",
    "I want to delete the recent booking",
]

In [ ]:
approach_one_data = list()

for i in range(0, 10):
    session_id:str = str(uuid.uuid1())
    turn = list()
    for message in messages:
        agent_answer, execution_time, input_tokens, output_tokens = invokeAgent(query=message, session_id=session_id, enable_trace=True, session_state=dict())
        turn.append({"latency": float(execution_time), "inputTokens": int(input_tokens), "outputTokens": int(output_tokens)})
        time.sleep(25) # for throttling
    
    approach_one_data.append(turn)


In [41]:
# the json file where the output must be stored
approach_one_data_json = open("approach_one_data.json", "w")

json.dump(approach_one_data, approach_one_data_json, indent = 6)

approach_one_data_json.close()

In [ ]:
approach_two_data = list()

for i in range(0, 10):
    session_id:str = str(uuid.uuid1())
    turn = list()
    conversation_history = list()

    for message in messages:
        session_id:str = str(uuid.uuid1())
        session_state = {'conversationHistory': {'messages': conversation_history}}
        agent_answer, execution_time, input_tokens, output_tokens = invokeAgent(query=message, session_id=session_id, enable_trace=True, session_state=session_state)
        turn.append({"latency": float(execution_time), "inputTokens": int(input_tokens), "outputTokens": int(output_tokens)})
    
        conversation_history.insert(0, {"content": [{"text": query}], "role": "user"})
        conversation_history.insert(0, {"content": [{"text": agent_answer}], "role": "assistant"})
        
        _, _, _, _ = invokeAgent(query="end", session_id=session_id, enable_trace=False, session_state=dict(), end_session=True)
        
        time.sleep(25) # for throttling

    
    approach_two_data.append(turn)

In [ ]:
# the json file where the output must be stored
approach_two_data_json = open("approach_two_data.json", "w")

json.dump(approach_two_data, approach_two_data_json, indent = 6)

approach_two_data_json.close()

In [52]:
with open('approach_one_data.json', 'r') as f:
    approach_one_data = json.load(f)

with open('approach_two_data.json', 'r') as f:
    approach_two_data = json.load(f)

In [ ]:
create_latency_boxplot(approach_one_data, approach_two_data)

In [ ]:
create_token_boxplot(approach_one_data, approach_two_data)

In [ ]:
last_message_analytics(approach_one_data, approach_two_data)

## Cleanup

In [ ]:
cleanup(agent_id, alias_id, kb_id, lambda_function, agent_action_group_response, knowledge_base)